# HLS Generate

After the `config.json` and `report.json` files are generated using the solver, the configuration is integrated into the HLS setup in [fpgaconvnet-hls](https://github.com/AlexMontgomerie/fpgaconvnet-hls).

To start with HLS generation, an instance of the network is generated by specifying the configuration file, ONNX file, and device information. In this example, we use the greedy partition algorithm, whose configuration file is generated in `4.2_optimiser_solver`. 

In [1]:
import onnx
import toml 
from fpgaconvnet.hls.generate.network import GenerateNetwork
from fpgaconvnet.parser.Parser import Parser

network_name = "vgg16_bn"
device_path = "../../fpgaconvnet-optimiser/examples/platforms/zcu104.toml"
onnx_path = "../3.1_model_onnx_parser/fp16/vgg16_bn.onnx"
# We choose greedy partition algorithm
config_path = "../4.2_optimiser_solver/gp/config.json"
with open(device_path, "r") as f:
    conf = toml.load(f)
    device_name = conf["device"]["part"]


# create instance of the network
gen_net = GenerateNetwork(network_name, config_path, onnx_path, device_name)

FIXME: use HLS backend in Parser
CRITICAL WARNING: node Flatten_31 is skipped in hardware


After the network is generated, we create an HLS project in Vivado HLS. [`create_partition_project(0)`](https://github.com/AlexMontgomerie/fpgaconvnet-hls/blob/dev/fpgaconvnet/hls/generate/network.py#L83) creates the HLS project, generates design files and testbench files, and includes them in the project. Note that the design currently doesn't support multithreading for the creation of HLS projects, so the hardware IP for each partition needs to be generated one by one. Here, we only create a project for partition 0.

`create_partition_project(0)` includes the following member functions:

- [`generate_layers()`](https://github.com/AlexMontgomerie/fpgaconvnet-hls/blob/main/fpgaconvnet/hls/generate/partition.py#L72): Utilizes the layer generator functions, detects the type of layer, and generates the header and source files using templates.

- [`generate_parameters()`](https://github.com/AlexMontgomerie/fpgaconvnet-hls/blob/main/fpgaconvnet/hls/generate/partition.py#L126): Retrieves parameters of the layer, generates weights from the ONNX model and saves them to a `.dat` file, and generates biases from the ONNX model and saves them to a `.csv` file.

- [`generate_streams()`](https://github.com/AlexMontgomerie/fpgaconvnet-hls/blob/main/fpgaconvnet/hls/generate/partition.py#L199): Defines streams for all layers in HLS.

- [`generate_include()`](https://github.com/AlexMontgomerie/fpgaconvnet-hls/blob/main/fpgaconvnet/hls/generate/partition.py#L216): Generates the top header from a template.

- [`generate_source()`](https://github.com/AlexMontgomerie/fpgaconvnet-hls/blob/main/fpgaconvnet/hls/generate/partition.py#L252): Generates the top source file from a template.

- [`generate_testbench`](https://github.com/AlexMontgomerie/fpgaconvnet-hls/blob/main/fpgaconvnet/hls/generate/partition.py#L279): Generates the C++ testbench from a template.

- [`create_vivado_hls_project`](https://github.com/AlexMontgomerie/fpgaconvnet-hls/blob/main/fpgaconvnet/hls/generate/partition.py#L325): Executes a system command to run `create_partition_project.tcl` to create a Vivado HLS project. 

In [2]:
gen_net.create_partition_project(0)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.1 (64-bit)
  **** SW Build 2552052 on Fri May 24 14:47:09 MDT 2019
  **** IP Build 2548770 on Fri May 24 18:01:18 MDT 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /tools/Xilinx/Vivado/2019.1/scripts/vivado_hls/hls.tcl -notrace
INFO: Applying HLS Y2K22 patch v1.2 for IP revision
INFO: [HLS 200-10] Running '/tools/Xilinx/Vivado/2019.1/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'lambert' on host 'lambert-ThinkPad-T14p-Gen-2' (Linux_x86_64 version 6.8.0-38-generic) on Tue Jul 16 16:17:08 BST 2024
INFO: [HLS 200-10] On os Ubuntu 24.04 LTS
INFO: [HLS 200-10] In directory '/home/lambert/Workspace/fpgaconvnet-tutorial/tutorial/5.1_hls_generate'
Sourcing Tcl script '/home/lambert/Workspace/fpgaconvnet-tutorial/fpgaconvnet-hls/fpgaconvnet/hls/scripts/hls/create_partition_project.tcl'
project: partition_0
INFO: [HLS 200-10] Creating and opening project '/home/lambert/Workspa

After the programs and data are added to the project, we run the simulation. Random testbench data is generated complying with the dimensional parameters, then [`run_testbench`](https://github.com/AlexMontgomerie/fpgaconvnet-hls/blob/dev/fpgaconvnet/hls/generate/network.py#L123), specifying the partition index and the testbench data, runs the simulation. Note that the RTL program hasn't been generated yet; the simulation aims to verify the functionality of the HLS programs. 

In [3]:
import numpy as np

input_image = np.random.rand(1,3,32,32).astype(np.float32)
# run C simulation
gen_net.run_testbench(0, input_image)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.1 (64-bit)
  **** SW Build 2552052 on Fri May 24 14:47:09 MDT 2019
  **** IP Build 2548770 on Fri May 24 18:01:18 MDT 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /tools/Xilinx/Vivado/2019.1/scripts/vivado_hls/hls.tcl -notrace
INFO: Applying HLS Y2K22 patch v1.2 for IP revision
INFO: [HLS 200-10] Running '/tools/Xilinx/Vivado/2019.1/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'lambert' on host 'lambert-ThinkPad-T14p-Gen-2' (Linux_x86_64 version 6.8.0-38-generic) on Tue Jul 16 16:17:11 BST 2024
INFO: [HLS 200-10] On os Ubuntu 24.04 LTS
INFO: [HLS 200-10] In directory '/home/lambert/Workspace/fpgaconvnet-tutorial/tutorial/5.1_hls_generate'
Sourcing Tcl script '/home/lambert/Workspace/fpgaconvnet-tutorial/fpgaconvnet-hls/fpgaconvnet/hls/scripts/hls/run_csim.tcl'
INFO: [HLS 200-10] Opening project '/home/lambert/Workspace/fpgaconvnet-tutorial/tutorial/5.1_hls_generate/

The hardware IP is generated by synthesizing the HLS programs. 

In [4]:
# run C synthesis and export IP for partition 0
gen_net.generate_partition_hardware(0)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.1 (64-bit)
  **** SW Build 2552052 on Fri May 24 14:47:09 MDT 2019
  **** IP Build 2548770 on Fri May 24 18:01:18 MDT 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /tools/Xilinx/Vivado/2019.1/scripts/vivado_hls/hls.tcl -notrace
INFO: Applying HLS Y2K22 patch v1.2 for IP revision
INFO: [HLS 200-10] Running '/tools/Xilinx/Vivado/2019.1/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'lambert' on host 'lambert-ThinkPad-T14p-Gen-2' (Linux_x86_64 version 6.8.0-38-generic) on Tue Jul 16 18:52:53 BST 2024
INFO: [HLS 200-10] On os Ubuntu 24.04 LTS
INFO: [HLS 200-10] In directory '/home/lambert/Workspace/fpgaconvnet-tutorial/tutorial/5.1_hls_generate'
Sourcing Tcl script '/home/lambert/Workspace/fpgaconvnet-tutorial/fpgaconvnet-hls/fpgaconvnet/hls/scripts/hls/run_csynth.tcl'
INFO: [HLS 200-10] Opening project '/home/lambert/Workspace/fpgaconvnet-tutorial/tutorial/5.1_hls_generat

After the synthesis is completed, we can view the resource utilization information of the generated hardware and compare it with the estimation in the model.

Five types of resources are tracked: FF, LUT, DSP, BRAM, and URAM. The resource analysis is conducted hierarchically, from partitions to layers and modules. Two types of models are applied: the [analytical model](https://github.com/AlexMontgomerie/fpgaconvnet-model/blob/dev/fpgaconvnet/tools/resource_analytical_model.py) and the [regression model](https://github.com/AlexMontgomerie/fpgaconvnet-model/blob/dev/fpgaconvnet/tools/resource_regression_model.py). Estimation uses the model based on the type of resources.

The resource utilization data in Vivado HLS is as follows: 

![resource_hls.png](./figures/resource_hls.png)

The resource utilization estimation data is as follows: 

![resource_estimation.png](./figures/resource_estimation.png)

There are clear discrepancies: in Vivado HLS, the resource utilization percentage exceeds 100%, which cannot happen, indicating that the model used in HLS may be incorrect. However, the only two resources where the two methods match are the number of DSP and URAM. The number of DSP is calculated as `coarse_in` * `coarse_out` * `fine`, with the terminologies explained [here](../4.1_optimiser_transform/RUNME.ipynb), while no URAM resources are used. 

Finally, [`run_cosimulation`](https://github.com/AlexMontgomerie/fpgaconvnet-hls/blob/dev/fpgaconvnet/hls/generate/network.py#L145) uses the C testbench to verify the generated RTL design. 

In [5]:
# run cosimulation
gen_net.run_cosimulation(0, input_image)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.1 (64-bit)
  **** SW Build 2552052 on Fri May 24 14:47:09 MDT 2019
  **** IP Build 2548770 on Fri May 24 18:01:18 MDT 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /tools/Xilinx/Vivado/2019.1/scripts/vivado_hls/hls.tcl -notrace
INFO: Applying HLS Y2K22 patch v1.2 for IP revision
INFO: [HLS 200-10] Running '/tools/Xilinx/Vivado/2019.1/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'lambert' on host 'lambert-ThinkPad-T14p-Gen-2' (Linux_x86_64 version 6.8.0-38-generic) on Tue Jul 16 22:50:42 BST 2024
INFO: [HLS 200-10] On os Ubuntu 24.04 LTS
INFO: [HLS 200-10] In directory '/home/lambert/Workspace/fpgaconvnet-tutorial/tutorial/5.1_hls_generate'
Sourcing Tcl script '/home/lambert/Workspace/fpgaconvnet-tutorial/fpgaconvnet-hls/fpgaconvnet/hls/scripts/hls/run_cosim.tcl'
INFO: [HLS 200-10] Opening project '/home/lambert/Workspace/fpgaconvnet-tutorial/tutorial/5.1_hls_generate

You may repeat the above steps for all partitions to generate the complete hardware design. The reason we need to generate the hardware design for each partition separately is that if all the partitions are generated in a single IP, the hardware doesn't have enough memory for the entire design. Therefore, the network is divided into multiple partitions, each containing a few different layers. 